## Проект: RND Exploratory Behavior

**Команда: Сергей Червонцев, Иван Провилков**


Мы исследуем exploration поведение агента используя "Random Network Distillation (RND)" (https://arxiv.org/abs/1810.12894). 

Интересно узнать, как ведет себя агент не имея награды от среды, когда он пользуется только своим внутренним "интересом", который обеспечивается с помощью RND. Меньше ли получается итоговое качество, сколько времени занимает такое обучение и какие факторы на него влияют?

### Мы поставили себе задачу написать агента, воспроизвести результаты статьи про RND на игре "Montezuma revenge" и посмотреть на поведение агента без награды от среды. 

### Мы обнаружили, что данная среда и алгоритм работают очень долго. В дальнейшем (после сдачи проекта) мы собираемся продолжать эксперименты с этой средой. Поэтому одной из своих главных задач на проект мы сделали ускорение обучения алгоритма в этой среде. Чтобы эксперименты занимали разумное время на наших ресурсах (2 сервера по 2 1080ti).

### Описание среды:

Montezuma revenge это игра Atari, в которой агент должен пройти через множество комнат (всего их 99) в лабиринте, чтобы найти сокровище. Определенные комнаты закрыты дверями и чтобы туда попасть нужно найти и подобрать ключ. В комнатах есть монстры, которые убивают персонажа и их нужно избегать. Также персонаж умирает от падения с высоты или попадания в ловушку (падение в лаву). В комнатах есть различные алмазы, которые игрок может подбирать и получить награду, а также снаряжение, которое можно надеть (факелы, мечи...). У персонажа есть 5 жизней, то есть он может 5 раз умереть до начала новой игры.

Состояния среды: RGB Картинка 210x160 пикселей (мы рескейлим до 84х84)

Пространство действий: агент может бегать в разные стороны и прыгать, всего 18 Discrete действий.

Extrinsic Reward: даётся за подбор ключа, открытие двери, подбор алмазов в комнатах.


### Модель

Мы воспроизводим модель из статьи "Random Network Distillation (RND)". Используем Pytorch. 

### Гиперпараметры:

Все гиперпараметры эксперимента можно посмотреть в config.yaml.

In [3]:
!cat config.yaml

EnvName: MontezumaRevengeNoFrameskip-v4
MaxStepsPerEpisode: 4500
ImageHeight: 84
ImageWidth: 84
UseStickyAction: True
StickyActionProb: 0.25

NumWorkers: 128
RolloutSteps: 128
NumInitSteps: 1024

ExtRewardDiscount: 0.999
IntRewardDiscount: 0.99
ExtCoeff: 2.0
IntCoeff: 1.0
LearningRate: 0.0001
ClipGradNorm: 10.0
BatchSize: 4096
EpochSteps: 4
SavePath: "/home/chervontsev/checkpoints/RndAgentAlt.ckpt"

NumEpochs: 10000

RNDUpdateProportion: 0.25
PPOEntropyCoeff: 0.001
PPORewardEps: 0.1
PPOAdvLambda: 0.95
UseVTraceCorrection: True

UseTPU: False


#### Некоторые гиперпараметры:

UseStickyAction: True -- использовать ли повторение предыдущего действия

StickyActionProb: 0.25 -- вероятность того, что повторится предыдущее действие

NumWorkers: 128 -- количество воркеров (процессов) в которых параллельно будет происходить сбор статистик для обучения. 

RolloutSteps: 128 -- количество шагов делаемых каждым воркером для сбора статистики

NumInitSteps: 1024 -- количество стартовых шагов.

ExtCoeff (extrinsic coefficient): -- коэффициент отвечающий за то как сильно мы учитываем награду от среды.

IntCoeff (intrinsic coefficient): -- коэффициент отвечающий за то как сильно мы учитываем внутреннюю награду (Random Network).

UseVTraceCorrection: -- использовать ли V-Trace

UseTPU -- в разработке

### Результаты запусков:

### 1) Ext Coeff: 2.0, Int Coeff 1.0

**ExtRew per Rollout:**
![](./log_screens/usual_2_reward_per_rollout.png)

**IntRew per Rollout:**
![](./log_screens/usual_2_int_reward_per_rollout.png)

**ExtRew per Episode (we log 1 worker with fixed index, total there are 128 workers):**
![](./log_screens/usual_2_reward_per_epi.png)

**Example of work (checkpoint from 5k steps):**
![](./videos/usual_training_2.gif)


### ExtCoeff: 1.0, IntCoeff: 1.0
#### Понижение влияния Extrinsic Reward (награды от среды)
TODO

### ExtCoeff 0.0, IntCoeff 1.0
#### Оставим только Intrinsic Reward (RND)

**ExtRew per Rollout:**
![](./log_screens/only_i_reward_per_rollout.png)

**IntRew per Rollout:**
![](./log_screens/only_i_int_reward_per_rollout.png)

**ExtRew per Episode (model does not train on it):**
![](./log_screens/only_i_reward_per_epi.png)

**Чекпоинт с этого запуска не сохранился, поэтому без видео**

#### Видим, что когда модель не наблюдает награду от среды, то эта награда увеличивается очень медленно (модель никто не поощряет за взятие алмазов). Также интересно то, что Intrinsic Reward после 3000 шагов начинает в среднем медленно расти. То есть модель понемногу учится исследовать что-то новое. 

### Ускорение

#### В базовой модели, которую мы написали, обучение на 10.000 шагов занимает около 3х дней. Причем дальнейшее ускорение вширь (увеличивая количество cpu или gpu) сделать трудно, так как увеличение NWorkers не ускоряет обучение моделей, как и увеличение размера сети или батча. Мы замерили, что в такой реализации на работу сбора статистик в environment-е тратится около 60% времени работы основного процесса (при том что этот сбор делается сразу 128 воркерами). В это время обучение сети на ГПУ не происходит. Мы решили распараллелить два этих процесса, чтобы обучение происходило одновременно со сбором статистик на старых (отстающих на 1 апдейт) весах сети (как в IMPALA).

**Результаты ускорения:**

### ExtCoeff 0.0, IntCoeff 1.0 (parallel env and training)

**ExtRew per Rollout:**
![](./log_screens/only_int_fast_reward_per_rollout.png)

**IntRew per Rollout:**
![](./log_screens/only_int_fast_int_reward_per_rollout.png)

**ExtRew per Episode (model does not train on it):**
![](./log_screens/only_int_fast_reward_per_epi.png)

**Example of work:**
![](./videos/only_intrinsic_fast.gif)

**На этом видео агент научился долго не умирать. При этом интересно, что он заходит во вторую комнату и начинает "играться с летающими черепами, а как-только их там не становится, то больше он туда не хочет заходить. Скорее всего это происходит потому, что прыгая вокруг черепов он постоянно получает новые картинки, не те, к которым он уже привык. За эти новые картинки intrinsic reward награждает его, так как агент не часто видел эти состояния и еще не успел выучить ответ случайной сети.**

**Также, судя по графикам обучения, то, что мы делаем подсчет статистик на старых (отстающих на 1 апдейт) весах влияет на обучение, потому что за то же время агент достиг меньшей награды.**

**Для того чтобы с этим побороться мы решили добавить V-Trace**

In [11]:
import torch

In [12]:
torch.__version__

'1.5.0+cu92'

## Инструкции по запуску кода

### Для запуска модели нужно установить нужные библиотеки, мы используем python3.6 и cuda9.2

In [14]:
!pip install -r requirements.txt

Установка torch

In [15]:
!pip install torch==1.5.0+cu92 torchvision==0.6.0+cu92 -f https://download.pytorch.org/whl/torch_stable.html

### Запуск агента с обученными весами

1. Скачиваем веса из google drive с помощью wget (ниже) или руками: https://drive.google.com/drive/folders/15RBD-BrWUlylYLR13u7NWRhnMTAb9cWs?usp=sharing
2. Пишем путь до весов в config.yaml. 
Пример: SavePath: "/home/chervontsev/checkpoints/usual_2.ckpt", выбираем надо ли оставлять UseStickyAction, мы по умолчанию оставляли. 
3. Запускаем eval.py скрипт: python eval.py, в логах скрипта: шаг с которого взят чекпоинт, ревард среды за игру, список посещенных комнат.
4. Видео игры агента появится в папке MontezumaRevengeNoFrameskip-v4_example_run в корне проекта.

In [22]:
# usual_2.ckpt
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1XyO39st3QnCg5NnQ__OTH-ohwWA93YAx' -O usual_2.ckpt

--2020-05-28 21:33:17--  https://docs.google.com/uc?export=download&id=1XyO39st3QnCg5NnQ__OTH-ohwWA93YAx
Resolving docs.google.com (docs.google.com)... 2a00:1450:4010:c01::c2, 64.233.162.194
Connecting to docs.google.com (docs.google.com)|2a00:1450:4010:c01::c2|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-00-18-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ddul2fqhs1g7fjud52ah7n359rau0rap/1590690750000/01498080638632719806/*/1XyO39st3QnCg5NnQ__OTH-ohwWA93YAx?e=download [following]
--2020-05-28 21:33:22--  https://doc-00-18-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ddul2fqhs1g7fjud52ah7n359rau0rap/1590690750000/01498080638632719806/*/1XyO39st3QnCg5NnQ__OTH-ohwWA93YAx?e=download
Resolving doc-00-18-docs.googleusercontent.com (doc-00-18-docs.googleusercontent.com)... 2a00:1450:4010:c02::84, 74.125.205.132
Connecting to doc-00-18-docs.googleusercontent.com (doc-00-18-docs.g

In [25]:
# only_intrinsic_fast.ckpt
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=16UteZvE_4bcciNiCB4jK7EG_h1crHLeo'\
    -O only_intrinsic_fast.ckpt

--2020-05-28 21:42:39--  https://docs.google.com/uc?export=download&id=16UteZvE_4bcciNiCB4jK7EG_h1crHLeo
Resolving docs.google.com (docs.google.com)... 2a00:1450:4010:c01::c2, 64.233.162.194
Connecting to docs.google.com (docs.google.com)|2a00:1450:4010:c01::c2|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0k-18-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/q8k7o0c761eha360s6i831rlp319ermp/1590691350000/01498080638632719806/*/16UteZvE_4bcciNiCB4jK7EG_h1crHLeo?e=download [following]
--2020-05-28 21:42:42--  https://doc-0k-18-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/q8k7o0c761eha360s6i831rlp319ermp/1590691350000/01498080638632719806/*/16UteZvE_4bcciNiCB4jK7EG_h1crHLeo?e=download
Resolving doc-0k-18-docs.googleusercontent.com (doc-0k-18-docs.googleusercontent.com)... 2a00:1450:4010:c02::84, 74.125.205.132
Connecting to doc-0k-18-docs.googleusercontent.com (doc-0k-18-docs.g

In [21]:
!python eval.py

N_updates 5000
Finished, total reward is 4600
All visited rooms: {0, 1, 4, 10, 11, 18, 19, 20}


**После удачного запуска агентов, когда мы убедились, что алгоритм работает, мы решили поставить себе задачу ускорить процесс обучения, так как обучение на этой среде занимает очень много времени и у нас не очень много ресурсов.**

**Этот процесс мы разделили на несколько этапов:**

1. После профилирования мы обнаружили, что много времени (~10 секунд на 1 .accumulate_rollout_steps() со 128 лернерами(воркерами)) занимает подготовка статистик необходимых для шага обучения агента, и примерно столько-же времени занимает сам шаг обучения. То есть ГПУ простаивает половину времени.Чтобы решить эту проблему мы решили сделать раздельное обучение actor-а (который собирает статистики необходимые для работы PPO) и learner-а (который делает апдейты оптимайзером) на примере того как это сделано в IMPALA (Включая V-trace).
2. Подумали, что стоит переписать имеющийся код на TPU.

**Мы реализовали пункт 1, что дало нам ускорение примерно в 2.3 раза. Результаты находятся в ветке impala (https://github.com/JanRocketMan/lightning_rnd/tree/impala)**

**Второй пункт остается на future work**

# Результаты

**Нам удалось воспроизвести статью Random Network Distillation.**

**Наши эксперименты показали, что с помощью RND агент может учиться без награды от среды. Однако, это занимает больше времени, а также ему труднее попасть в новые комнаты, в которые он как-то попал в прошлый раз. (Так как нет сильного дополнительного реварда от среды).**

**Мы реализовали оптимизацию, которая разделяет обучение actor-а и learner-а, что дало нам ускорение в 2.3 раза.**